<a href="https://colab.research.google.com/github/pravallika-73/image-classification-project/blob/main/image_classification(animals).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PABBU'S FIXED CAT vs DOG CLASSIFIER (Uses TFDS – No Download Errors!)

import tensorflow as tf
import tensorflow_datasets as tfds
print("GPU:", tf.config.list_physical_devices('GPU'))
print("TensorFlow:", tf.__version__)

# 1. Load dataset using TensorFlow Datasets (automatic download & prep)
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[20%:]'],  # 80/20 split
    with_info=True,
    as_supervised=True,  # Returns (image, label)
    shuffle_files=True
)

# 2. Preprocess function (resize, normalize)
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    image = tf.image.resize(image, (150, 150))   # Resize
    return image, label

# Apply preprocessing
ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

print(f"Train batches: {len(ds_train)}, Val batches: {len(ds_val)}")
print(f"Classes: {ds_info.features['label'].names}")  # Should show ['cats', 'dogs']

# 3. Imports for model
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
from google.colab import files

# 4. Build the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary: cat=0, dog=1
])

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)
model.summary()

# 5. Train the model (8-12 min on GPU)
history = model.fit(
    ds_train,
    epochs=40,
    validation_data=ds_val,
    callbacks=[
        EarlyStopping(patience=7, restore_best_weights=True),
        ModelCheckpoint('best_model.keras', save_best_only=True)
    ]
)

# 6. Plot results
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()

# 7. Download the model
files.download('best_model.keras')

# 8. Test on your own photo
print("Upload a cat or dog photo to test!")
uploaded = files.upload()
class_names = ['CAT', 'DOG']  # 0=cat, 1=dog
for fn in uploaded.keys():
    img = image.load_img(fn, target_size=(150,150))
    x = np.expand_dims(image.img_to_array(img), axis=0) / 255.0
    pred = model.predict(x)[0][0]
    label = class_names[1] if pred > 0.5 else class_names[0]
    conf = pred * 100 if pred > 0.5 else (1 - pred) * 100
    plt.imshow(img)
    plt.title(f"PREDICTION: {label} ({conf:.1f}% confidence)")
    plt.axis('off')
    plt.show()

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow: 2.19.0


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 58s 89ms/step - accuracy: 0.5634 - loss: 0.6790 - val_accuracy: 0.7197 - val_loss: 0.5649
Epoch 2/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 66s 113ms/step - accuracy: 0.7182 - loss: 0.5541 - val_accuracy: 0.6968 - val_loss: 0.5531
Epoch 3/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 46s 78ms/step - accuracy: 0.7746 - loss: 0.4741 - val_accuracy: 0.8264 - val_loss: 0.3875
Epoch 4/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 47s 81ms/step - accuracy: 0.8165 - loss: 0.4072 - val_accuracy: 0.8014 - val_loss: 0.4287
Epoch 5/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - accuracy: 0.8483 - loss: 0.3494 - val_accuracy: 0.8677 - val_loss: 0.3262
Epoch 6/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - accuracy: 0.8726 - loss: 0.3010 - val_accuracy: 0.9105 - val_loss: 0.2201
Epoch 7/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 46s 79ms/step - accuracy: 0.8902 - loss: 0.2619 - val_accuracy: 0.9105 - val_loss: 0.2212
Epoch 8/40
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 79ms/step - accuracy: 0.9011 - loss: 0.2365 -